In [1]:
import sys
import numpy as np
sys.path.append('../')
from deepod.models.time_series import COUTA, TimesNet, DeepSVDDTS,TranAD,AnomalyTransformer,USAD,DeepIsolationForestTS,GDeepSVDDTS
from deepod.metrics import ts_metrics
from deepod.metrics import point_adjustment 


In [21]:
import sys
import numpy as np
import pandas as pd
import random
import torch
from deepod.models.time_series import (
    COUTA, TimesNet, DeepSVDDTS, TranAD, 
    AnomalyTransformer, USAD, GDeepSVDDTS
)
from deepod.metrics import ts_metrics, point_adjustment

# Path setup
sys.path.append('../')

# Data loading function
def load_data(dataset):
    if dataset == 0:
        train_data = np.load('../../../TranAD/processed/SWaT/train.npy')
        test_data = np.load('../../../TranAD/processed/SWaT/test.npy')
        labels = np.load('../../../TranAD/processed/SWaT/labels.npy')
    elif dataset == 1:
        train_data = np.load('/home/iot/GSVDD/TranAD/processed/SMAP/P-1_train.npy')
        test_data = np.load('/home/iot/GSVDD/TranAD/processed/SMAP/P-1_test.npy')
        labels = np.load('/home/iot/GSVDD/TranAD/processed/SMAP/P-1_labels.npy')[:, 0]
    elif dataset == 2:
        train_data = np.load('/home/iot/GSVDD/TranAD/processed/MSL/C-1_train.npy')
        test_data = np.load('/home/iot/GSVDD/TranAD/processed/MSL/C-1_test.npy')
        labels = np.load('/home/iot/GSVDD/TranAD/processed/MSL/C-1_labels.npy')[:, 0]
    return train_data, test_data, labels

# Function to set random state for reproducibility
def set_random_state(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

# List of models to evaluate
models = {
    "DeepSVDDTS": lambda seed: DeepSVDDTS(network='TCN',random_state=seed),
    "USAD": lambda seed: USAD(random_state=seed),
    "DeepIsolationForestTS":lambda seed: DeepIsolationForestTS(random_state=seed),
    # "TimesNet": lambda seed: TimesNet(random_state=seed),
    "TranAD": lambda seed: TranAD(random_state=seed),
    "COUTA": lambda seed: COUTA(random_state=seed),
    "AnomalyTransformer": lambda seed: AnomalyTransformer(random_state=seed),
    "GDeepSVDDTS": lambda seed: GDeepSVDDTS(epochs=70,random_state=seed)
}

# Results storage
results_table = []

# Repeat for 3 datasets
for dataset_idx in range(3):
    train_data, test_data, labels = load_data(dataset_idx)
    print(f"Dataset {dataset_idx}: Running experiments...")

    # Train, evaluate, and repeat 10 times for all models
    for model_name, model_factory in models.items():
        print(f"Running model {model_name}...")
        raw_metrics_list = []
        adj_metrics_list = []
        
        for run in range(10):
            seed = 42 + dataset_idx * 10 + run  # Seed for reproducibility
            set_random_state(seed)

            print(f"Run {run + 1}/10 for {model_name} with Seed {seed}...")
            model = model_factory(seed)
            model.fit(train_data)

            # Prediction and evaluation
            scores = model.decision_function(test_data)
            raw_eval_metrics = ts_metrics(labels, scores)
            adjusted_metrics = ts_metrics(labels, point_adjustment(labels, scores))
            print(adjusted_metrics)
            # Collect raw and adjusted metrics
            raw_metrics_list.append([
                raw_eval_metrics[0],  # Raw AUC
                raw_eval_metrics[2],  # Raw F1
                raw_eval_metrics[3],  # Raw Precision
                raw_eval_metrics[4],  # Raw Recall
            ])
            adj_metrics_list.append([
                adjusted_metrics[0],  # Adjusted AUC
                adjusted_metrics[2],  # Adjusted F1
                adjusted_metrics[3],  # Adjusted Precision
                adjusted_metrics[4],  # Adjusted Recall
            ])

        # Calculate mean and standard deviation for both raw and adjusted metrics
        raw_metrics_array = np.array(raw_metrics_list)
        adj_metrics_array = np.array(adj_metrics_list)

        raw_means, raw_stds = np.mean(raw_metrics_array, axis=0), np.std(raw_metrics_array, axis=0)
        adj_means, adj_stds = np.mean(adj_metrics_array, axis=0), np.std(adj_metrics_array, axis=0)

        # Append the results for table format
        results_table.append([
            f"Dataset {dataset_idx}",
            model_name,
            f"{raw_means[0]:.3f} ± {raw_stds[0]:.3f}",  # Raw AUC
            f"{raw_means[1]:.3f} ± {raw_stds[1]:.3f}",  # Raw F1
            f"{raw_means[2]:.3f} ± {raw_stds[2]:.3f}",  # Raw Precision
            f"{raw_means[3]:.3f} ± {raw_stds[3]:.3f}",  # Raw Recall
            f"{adj_means[0]:.3f} ± {adj_stds[0]:.3f}",  # Adjusted AUC
            f"{adj_means[1]:.3f} ± {adj_stds[1]:.3f}",  # Adjusted F1
            f"{adj_means[2]:.3f} ± {adj_stds[2]:.3f}",  # Adjusted Precision
            f"{adj_means[3]:.3f} ± {adj_stds[3]:.3f}",  # Adjusted Recall
        ])

# Convert results to DataFrame
columns = [
    "Dataset", "Model", 
    "Raw AUC (Mean ± Std)", "Raw F1 (Mean ± Std)", 
    "Raw Precision (Mean ± Std)", "Raw Recall (Mean ± Std)",
    "Adjusted AUC (Mean ± Std)", "Adjusted F1 (Mean ± Std)", 
    "Adjusted Precision (Mean ± Std)", "Adjusted Recall (Mean ± Std)"
]
df_results = pd.DataFrame(results_table, columns=columns)

# Save results to a CSV file
df_results.to_csv("time_series_anomaly_metrics_summary_deep.csv", index=False)

# Display the table
print("\nResults Summary (Mean ± Std):")
print(df_results.to_string(index=False))


Dataset 0: Running experiments...
Running model DeepSVDDTS...
Run 1/10 for DeepSVDDTS with Seed 42...
Start Training...
ensemble size: 1
{'n_features': 1, 'n_hidden': '512', 'n_output': 64, 'activation': 'GELU', 'bias': False}
TCNnet(
  (network): Sequential(
    (0): TcnResidualBlock(
      (conv1): Conv1d(1, 512, kernel_size=(2,), stride=(1,), padding=(1,), bias=False)
      (chomp1): Chomp1d()
      (act1): GELU(approximate='none')
      (dropout1): Dropout(p=0.2, inplace=False)
      (conv2): Conv1d(512, 512, kernel_size=(2,), stride=(1,), padding=(1,), bias=False)
      (chomp2): Chomp1d()
      (act2): GELU(approximate='none')
      (dropout2): Dropout(p=0.2, inplace=False)
      (net): Sequential(
        (0): Conv1d(1, 512, kernel_size=(2,), stride=(1,), padding=(1,), bias=False)
        (1): Chomp1d()
        (2): GELU(approximate='none')
        (3): Dropout(p=0.2, inplace=False)
        (4): Conv1d(512, 512, kernel_size=(2,), stride=(1,), padding=(1,), bias=False)
        (5

/home/iot/anaconda3/envs/compare/lib/python3.8/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


epoch 40, training loss: 0.003475, time: 0.0s
epoch 50, training loss: 0.003058, time: 0.0s
epoch 60, training loss: 0.002689, time: 0.0s
epoch 70, training loss: 0.002394, time: 0.0s
epoch 80, training loss: 0.002397, time: 0.0s
epoch 90, training loss: 0.002166, time: 0.0s
epoch100, training loss: 0.002120, time: 0.0s
Start Inference on the training data...


testing: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████| 78/78 [00:00<00:00, 2763.86it/s]

(0.9146640031027383, 0.8208955915660053, 0.8150825674961042, 1.0, 0.687888198757764)
Run 2/10 for DeepSVDDTS with Seed 43...
Start Training...
ensemble size: 1
{'n_features': 1, 'n_hidden': '512', 'n_output': 64, 'activation': 'GELU', 'bias': False}
TCNnet(
  (network): Sequential(
    (0): TcnResidualBlock(
      (conv1): Conv1d(1, 512, kernel_size=(2,), stride=(1,), padding=(1,), bias=False)
      (chomp1): Chomp1d()
      (act1): GELU(approximate='none')
      (dropout1): Dropout(p=0.2, inplace=False)
      (conv2): Conv1d(512, 512, kernel_size=(2,), stride=(1,), padding=(1,), bias=False)
      (chomp2): Chomp1d()
      (act2): GELU(approximate='none')
      (dropout2): Dropout(p=0.2, inplace=False)
      (net): Sequential(
        (0): Conv1d(1, 512, kernel_size=(2,), stride=(1,), padding=(1,), bias=False)
        (1): Chomp1d()
        (2): GELU(approximate='none')
        (3): Dropout(p=0.2, inplace=False)
        (4): Conv1d(512, 512, kernel_size=(2,), stride=(1,), padding=(1,),

...

epoch 60, training loss: 31.999231, time: 0.2s


/home/iot/anaconda3/envs/compare/lib/python3.8/site-packages/sklearn/covariance/_robust_covariance.py:747: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn(
/home/iot/anaconda3/envs/compare/lib/python3.8/site-packages/sklearn/covariance/_robust_covariance.py:747: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn(
/home/iot/anaconda3/envs/compare/lib/python3.8/site-packages/sklearn/covariance/_robust_covariance.py:747: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn(
/home/iot/anaconda3/envs/compare/lib/python3.8/site-packages/sklearn/covariance/_robust_covariance.py:747: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn(
/home/iot/anaconda3/envs/compare/lib/python3.8/site-packages/sklearn/covariance/_robust_covariance.py:747: UserWarning: The covariance matrix associated to your dataset is not full rank
  

epoch 70, training loss: 31.990101, time: 0.2s
Start Inference on the training data...


testing: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 35/35 [00:00<00:00, 848.95it/s]

(0.9965496747779575, 0.9795077948818511, 0.970261045134838, 0.9422492401215805, 1.0)

Results Summary (Mean ± Std):
  Dataset                 Model Raw AUC (Mean ± Std) Raw F1 (Mean ± Std) Raw Precision (Mean ± Std) Raw Recall (Mean ± Std) Adjusted AUC (Mean ± Std) Adjusted F1 (Mean ± Std) Adjusted Precision (Mean ± Std) Adjusted Recall (Mean ± Std)
Dataset 0            DeepSVDDTS        0.857 ± 0.008       0.775 ± 0.004              0.937 ± 0.011           0.660 ± 0.009             0.924 ± 0.011            0.815 ± 0.001                   0.986 ± 0.036                0.696 ± 0.024
Dataset 0                  USAD        0.788 ± 0.010       0.651 ± 0.001              0.646 ± 0.001           0.655 ± 0.002             0.804 ± 0.011            0.706 ± 0.002                   0.715 ± 0.004                0.697 ± 0.000
Dataset 0 DeepIsolationForestTS        0.812 ± 0.005       0.676 ± 0.000              0.657 ± 0.001           0.697 ± 0.000             0.845 ± 0.005            0.778 ± 0.006  

In [10]:
import sys
import numpy as np
import pandas as pd
from deepod.models.time_series import (
    COUTA, TimesNet, DeepSVDDTS, TranAD, 
    AnomalyTransformer, USAD, DeepIsolationForestTS
)
from deepod.metrics import ts_metrics, point_adjustment

# Path setup
sys.path.append('../')

# Data loading function
def load_data(dataset):
    if dataset == 0:
        train_data = np.load('../../../TranAD/processed/SWaT/train.npy')
        test_data = np.load('../../../TranAD/processed/SWaT/test.npy')
        labels = np.load('../../../TranAD/processed/SWaT/labels.npy')
    elif dataset == 1:
        train_data = np.load('/home/iot/GSVDD/TranAD/processed/SMAP/P-1_train.npy')
        test_data = np.load('/home/iot/GSVDD/TranAD/processed/SMAP/P-1_test.npy')
        labels = np.load('/home/iot/GSVDD/TranAD/processed/SMAP/P-1_labels.npy')[:, 0]
    elif dataset == 2:
        train_data = np.load('/home/iot/GSVDD/TranAD/processed/MSL/C-1_train.npy')
        test_data = np.load('/home/iot/GSVDD/TranAD/processed/MSL/C-1_test.npy')
        labels = np.load('/home/iot/GSVDD/TranAD/processed/MSL/C-1_labels.npy')[:, 0]
    return train_data, test_data, labels

# List of models to evaluate
models = {
    # "DeepSVDDTS": DeepSVDDTS(network='TCN'),
    "DeepIsolationForestTS": DeepIsolationForestTS(),
    # "USAD": USAD(),
    # "TimesNet": TimesNet(),
    # "TranAD": TranAD(),
    # "COUTA": COUTA(),
    # "AnomalyTransformer": AnomalyTransformer(),
    # "GDeepSVDDTS":GDeepSVDDTS(epochs=70)
}

# Results storage
results_summary = []

# Repeat for 3 datasets
for dataset_idx in range(0,3):
    train_data, test_data, labels = load_data(dataset_idx)
    print(f"Dataset {dataset_idx}: Running experiments...")

    # Train, evaluate, and repeat 5 times for all models
    for model_name, model in models.items():
        print(f"Running model {model_name}...")
        metrics_list = []

        for run in range(10):
            print(f"Run {run + 1}/5 for {model_name}...")
            model.fit(train_data)

            # Prediction and evaluation
            scores = model.decision_function(test_data)
            eval_metrics = ts_metrics(labels, scores)
            adj_metrics = ts_metrics(labels, point_adjustment(labels, scores))
            
            # Extract metrics of interest (AUC, F1, Precision, Recall)
            metrics_list.append([
                adj_metrics[0],  # AUC
                adj_metrics[2],  # F1
                adj_metrics[3],  # Precision
                adj_metrics[4],  # Recall
            ])
        
        # Calculate mean and standard deviation of metrics
        metrics_array = np.array(metrics_list)
        means = np.mean(metrics_array, axis=0)
        stds = np.std(metrics_array, axis=0)

        # Store results for each model and dataset
        results_summary.append({
            "Dataset": dataset_idx,
            "Model": model_name,
            "AUC Mean": means[0], "AUC Std": stds[0],
            "F1 Mean": means[1], "F1 Std": stds[1],
            "Precision Mean": means[2], "Precision Std": stds[2],
            "Recall Mean": means[3], "Recall Std": stds[3],
        })

# Save results to a CSV file
df_results = pd.DataFrame(results_summary)
df_results.to_csv("time_series_anomaly_results_2.csv", index=False)

# Display a preview of the results



Dataset 0: Running experiments...
Running model DeepSVDDTS...
Run 1/5 for DeepSVDDTS...
Start Training...
ensemble size: 1
{'n_features': 1, 'n_hidden': '512', 'n_output': 64, 'activation': 'GELU', 'bias': False}
TCNnet(
  (network): Sequential(
    (0): TcnResidualBlock(
      (conv1): Conv1d(1, 512, kernel_size=(2,), stride=(1,), padding=(1,), bias=False)
      (chomp1): Chomp1d()
      (act1): GELU(approximate='none')
      (dropout1): Dropout(p=0.2, inplace=False)
      (conv2): Conv1d(512, 512, kernel_size=(2,), stride=(1,), padding=(1,), bias=False)
      (chomp2): Chomp1d()
      (act2): GELU(approximate='none')
      (dropout2): Dropout(p=0.2, inplace=False)
      (net): Sequential(
        (0): Conv1d(1, 512, kernel_size=(2,), stride=(1,), padding=(1,), bias=False)
        (1): Chomp1d()
        (2): GELU(approximate='none')
        (3): Dropout(p=0.2, inplace=False)
        (4): Conv1d(512, 512, kernel_size=(2,), stride=(1,), padding=(1,), bias=False)
        (5): Chomp1d()
 

/home/iot/anaconda3/envs/compare/lib/python3.8/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


epoch 40, training loss: 0.003475, time: 0.0s
epoch 50, training loss: 0.003058, time: 0.0s
epoch 60, training loss: 0.002689, time: 0.0s
epoch 70, training loss: 0.002394, time: 0.0s
epoch 80, training loss: 0.002397, time: 0.0s
epoch 90, training loss: 0.002166, time: 0.0s
epoch100, training loss: 0.002120, time: 0.0s
Start Inference on the training data...


testing: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 78/78 [00:00<00:00, 3091.60it/s]

Run 2/5 for DeepSVDDTS...
Start Training...
ensemble size: 1
{'n_features': 1, 'n_hidden': '512', 'n_output': 64, 'activation': 'GELU', 'bias': False}
TCNnet(
  (network): Sequential(
    (0): TcnResidualBlock(
      (conv1): Conv1d(1, 512, kernel_size=(2,), stride=(1,), padding=(1,), bias=False)
      (chomp1): Chomp1d()
      (act1): GELU(approximate='none')
      (dropout1): Dropout(p=0.2, inplace=False)
      (conv2): Conv1d(512, 512, kernel_size=(2,), stride=(1,), padding=(1,), bias=False)
      (chomp2): Chomp1d()
      (act2): GELU(approximate='none')
      (dropout2): Dropout(p=0.2, inplace=False)
      (net): Sequential(
        (0): Conv1d(1, 512, kernel_size=(2,), stride=(1,), padding=(1,), bias=False)
        (1): Chomp1d()
        (2): GELU(approximate='none')
        (3): Dropout(p=0.2, inplace=False)
        (4): Conv1d(512, 512, kernel_size=(2,), stride=(1,), padding=(1,), bias=False)
        (5): Chomp1d()
        (6): GELU(approximate='none')
        (7): Dropout(p=0.

...

/home/iot/anaconda3/envs/compare/lib/python3.8/site-packages/sklearn/covariance/_robust_covariance.py:747: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn(
/home/iot/anaconda3/envs/compare/lib/python3.8/site-packages/sklearn/covariance/_robust_covariance.py:747: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn(
/home/iot/anaconda3/envs/compare/lib/python3.8/site-packages/sklearn/covariance/_robust_covariance.py:747: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn(
/home/iot/anaconda3/envs/compare/lib/python3.8/site-packages/sklearn/covariance/_robust_covariance.py:747: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn(
/home/iot/anaconda3/envs/compare/lib/python3.8/site-packages/sklearn/covariance/_robust_covariance.py:747: UserWarning: The covariance matrix associated to your dataset is not full rank
  

epoch 60, training loss: 31.991587, time: 0.1s


/home/iot/anaconda3/envs/compare/lib/python3.8/site-packages/sklearn/covariance/_robust_covariance.py:747: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn(
/home/iot/anaconda3/envs/compare/lib/python3.8/site-packages/sklearn/covariance/_robust_covariance.py:747: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn(
/home/iot/anaconda3/envs/compare/lib/python3.8/site-packages/sklearn/covariance/_robust_covariance.py:747: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn(
/home/iot/anaconda3/envs/compare/lib/python3.8/site-packages/sklearn/covariance/_robust_covariance.py:747: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn(
/home/iot/anaconda3/envs/compare/lib/python3.8/site-packages/sklearn/covariance/_robust_covariance.py:747: UserWarning: The covariance matrix associated to your dataset is not full rank
  

epoch 70, training loss: 31.985240, time: 0.1s
Start Inference on the training data...


testing: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 35/35 [00:00<00:00, 630.99it/s]
